# 方策の混合を利用した解法？

気になるやり方を見つけたのでメモ

参考：
* [Reinforcement Learning with Convex Constraints](https://proceedings.neurips.cc/paper_files/paper/2019/hash/873be0705c80679f2c71fbf4d872df59-Abstract.html)
* [Batch Policy Learning under Constraints](https://proceedings.mlr.press/v97/le19a.html)
* [Distributionally Robust Constrained Reinforcement Learning under Strong Duality](https://arxiv.org/abs/2406.15788)


## [Reinforcement Learning with Convex Constraints](https://proceedings.neurips.cc/paper_files/paper/2019/hash/873be0705c80679f2c71fbf4d872df59-Abstract.html)

準備：
* MDP：$M=\left(\mathcal{S}, \mathcal{A}, \beta, P_s, P_z\right)$．ここで，$P_s$は繊維確率．$P_z$は$d$次元のベクトル$\mathbf{z}_i \in \mathbb{R}^d$を返す分布．$\mathbf{z}_i \sim P_z\left(\cdot \mid s_i, a_i\right)$みたいな感じ．

$$
\overline{\mathbf{z}}(\pi) \triangleq \mathbb{E}\left[\sum_{i=0}^{\infty} \gamma^i \mathbf{z}_i \mid \pi\right]
$$
とする．

---

**Mixed policyについて**

以降では有限個の方策の集合$\Pi$に対して，$\Pi$上の有限個の方策を選択し，それ上の分布$\mu$を考えることにします．
例えば$\Pi$はすべての決定的な方策の集合などですね．
つまり，$\mu \in \Delta (\Pi)$です．
$\mu$をデプロイする際は，あるエピソードにおいて，ランダムに$\mu$から$\pi$を選択し，そのエピソード中はずっとその$\pi$を使うことになります．

ここで，その$\mu$に従って得られる期待ベクトルを
$$
\overline{\mathbf{z}}(\mu) \triangleq \mathbb{E}_{\pi \sim \mu}[\overline{\mathbf{z}}(\pi)]=\sum \mu(\pi) \overline{\mathbf{z}}(\pi)
$$
と表記します．

---

以上の表記を使って，実行可能解を次のように定義します：

$$
\text { Feasibility Problem: Find } \mu \in \Delta(\Pi) \text { such that } \overline{\mathbf{z}}(\mu) \in \mathcal{C} \text {. }
$$

これを考えてもいいですが，最適化問題として定式化しましょう：

$$
\min _{\mu \in \Delta(\Pi)} \operatorname{dist}(\overline{\mathbf{z}}(\mu), \mathcal{C})
$$

つまり，$\overline{\mathbf{z}}(\mu)$と$\mathcal{C}$のユークリッド距離が最も近くなるような$\mu$を見つける問題ですね．

ここで，例えば凸錐$\mathcal{C}$に対して
$$
\mathcal{C}^{\circ} \triangleq\{\boldsymbol{\lambda}: \boldsymbol{\lambda} \cdot \mathrm{x} \leq 0 \text { for all } \mathrm{x} \in \mathcal{C}\}
$$
なるPolar Coneを考えると，次が成り立ちます（証明は論文のLemma 3.2）：
$$
\operatorname{dist}(\mathbf{x}, \mathcal{C})=\max _{\boldsymbol{\lambda} \in \mathcal{C}^{\circ} \cap \mathcal{B}} \boldsymbol{\lambda} \cdot \mathbf{x}
$$

こんな感じの表現を使えば，元の問題は
$$
\min _{\mu \in \Delta(\Pi)} \max _{\boldsymbol{\lambda} \in \Lambda} \boldsymbol{\lambda} \cdot \overline{\mathbf{z}}(\mu)
$$
と等価です．最適化関数が明らかに線形なので，強双対性が成り立ちます．

あとはBest-responseアルゴリズム（MDPで最適方策を返すやつ）を$\mu$側に適用し，とNo-regretアルゴリズム（sublinearリグレットを達成するオンラインアルゴリズム）を$\lambda$側に適用すれば解けます．

**注釈：この定式化はそもそもありなのか？**

* 「あるエピソードにおいて，ランダムに$\mu$から$\pi$を選択し，そのエピソード中はずっとその$\pi$を使うことになります．」
これをすると，$\Pi$が全部決定的な方策の場合に絶対に制約を満たさない方策が出てくる．
* また，$\mu$を使って得られるリターン
（$\overline{\mathbf{z}}(\mu) \triangleq \mathbb{E}_{\pi \sim \mu}[\overline{\mathbf{z}}(\pi)]=\sum \mu(\pi) \overline{\mathbf{z}}(\pi)$）
と，$\mu$を使って得られる方策が出すリターンって同じか？リターンは$\pi$について凸じゃないので，同じにはならないと思う．
  * 例えば$\Pi=\{\pi_1, \pi_2\}$で，$\mu$がその２つを等しく選択するとしよう．すると，$\pi_1$と$\pi_2$を別エピソードで選択する場合は$\overline{\mathbf{z}}(\mu) = 0.5 [\pi_1] + 0.5 [\pi_2]$になるが，$0.5 \pi_1 + 0.5 \pi_2$のリターンはそれと等しいわけではないはず（非凸なので）．
* 結論：これは方策のIdentificationには使えなさそう．
